<a href="https://colab.research.google.com/github/ShaelinN/VAE_RECOMMENDER_IMPLICIT_FEEDBACK/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#All imports

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
from scipy import sparse
import pickle
import random

#Common directories

In [ ]:
root = "/content/drive/MyDrive/COMP700_Honours Project"

In [ ]:
data = os.path.join(root,"Data")

In [ ]:
specific_version_data = os.path.join(data,"ml20m")

In [ ]:
records_filename = "ml20m_reviews.csv"

In [ ]:
split = os.path.join(specific_version_data,"split")

In [ ]:
matrix_folder = os.path.join(split,"matrices")
implicit = os.path.join(matrix_folder, "implicit")
explicit = os.path.join(matrix_folder, "explicit")

In [ ]:
try:
  os.mkdir(specific_version_data)
except FileExistsError:
  pass

In [ ]:
try:
  os.mkdir(split)
except FileExistsError:
  pass

In [ ]:
try:
  os.mkdir(matrix_folder)
except FileExistsError:
  pass

try:
  os.mkdir(implicit)
except FileExistsError:
  pass

try:
  os.mkdir(explicit)
except FileExistsError:
  pass


In [ ]:
records = pd.read_csv(os.path.join(data , records_filename))
records.head()

,uid,bid,rate
0,u1u,b2b,3.5
1,u1u,b29b,3.5
2,u1u,b32b,3.5
3,u1u,b47b,3.5
4,u1u,b50b,3.5


#HACK
prevents pandas from changing datatypesto numeric and affecting the rest of the code

##only use if the uids and bids are of number format instead of text


In [ ]:
u = np.str('u')
b = np.str('b')

In [ ]:
str_u = [] 

In [ ]:
for i in range(len(records['uid'])):
  str_u.append(u+np.str(records['uid'][i])+u)

  if not i%10000:
    print(str_u[i], i)
#str_u = np.array(str_u)

In [ ]:
records['uid'] = str_u

In [ ]:
records.to_csv(os.path.join(root,"temprev.csv"), index=False)

In [ ]:
str_b = [] 

In [ ]:
for i in range(len(records['bid'])):
  str_b.append(b+np.str(records['bid'][i])+b)

  if not i%10000:
    print(str_b[i], i)

In [ ]:
records['bid'] = str_b

In [ ]:
str_b

In [ ]:
records.to_csv(os.path.join(data,records_filename), index=False)

#Filtration:
remove from the dataset users and businesses who do not have many interactions

###Filter
in line with other projects, remove records involving businesses with fewer than 5 reviews to them, (or involving users with fewer than 1 reviews by them)

In [ ]:
user_threshold = 5
business_threshold = 5

In [ ]:
def count(df, id):
  groupbyid = df[[id]].groupby(id, as_index=False)
  count = groupbyid.size()
  return count


In [ ]:
def filter(df, col, thresh):
  print("original:",df.shape)

  itemcount = count(df, col) # id:num occurrence of id
  retain_ids = itemcount.where(itemcount['size'] >= thresh).dropna() #which ids to keep
  print("unique col in final:",retain_ids.shape) #number of users in final


  where_idx = df[col].isin(retain_ids[col]) # original df row_idx: <true/false>(is col id at index in retain_ids?) same len as original df

  filtered = df.where(where_idx).dropna()#.astype(np.int64) #d
  print("final:",filtered.shape)



  uu = filtered['uid'].unique().shape[0]
  ub = filtered['bid'].unique().shape[0]
  items = filtered['rate'].shape[0]

  sparsity = (1.0 * items)/(uu * ub)
  print("After filtering, there are %d events from %d users and %d businesses (sparsity: %.3f%%)" % 
      (items, uu, ub, sparsity * 100))

  return filtered

In [ ]:
records = filter(records, "uid", user_threshold)
records = filter(records, "bid", business_threshold)

original: (20000263, 3)
unique col in final: (138493, 2)
final: (20000263, 3)
After filtering, there are 20000263 events from 138493 users and 26744 businesses (sparsity: 0.540%)
original: (20000263, 3)
unique col in final: (18345, 2)
final: (19984024, 3)
After filtering, there are 19984024 events from 138493 users and 18345 businesses (sparsity: 0.787%)


In [ ]:
records.head()

,uid,bid,rate
0,u1u,b2b,3.5
1,u1u,b29b,3.5
2,u1u,b32b,3.5
3,u1u,b47b,3.5
4,u1u,b50b,3.5


###saving:
 * the csv of filtered reviews
 * a shuffled order list of user ids and business ids


In [ ]:
records.to_csv(os.path.join(specific_version_data , "filtered_reviews.csv"), index=False)

In [ ]:
seed = np.random.seed(12345)
unique_u = records['uid'].unique()
np.random.shuffle(unique_u)

unique_b = records['bid'].unique()
np.random.shuffle(unique_b)
with open(os.path.join(specific_version_data,"unique_b.txt") , 'w') as bfile:
  bfile.writelines('\n'.join(unique_b))

with open(os.path.join(specific_version_data,"unique_u.txt") , 'w') as ufile:
  ufile.writelines('\n'.join(unique_u))

#Binarize Data Format
convert the reviews from a scale of 1-5, to a binary of 0-1. 
* [1..3] -> 0 
* [4..5] -> 1

In [ ]:
#records = pd.read_csv(os.path.join(specific_version_data , "filtered_reviews.csv"))
records.head()

,uid,bid,rate
0,u1u,b2b,3.5
1,u1u,b29b,3.5
2,u1u,b32b,3.5
3,u1u,b47b,3.5
4,u1u,b50b,3.5


In [ ]:
records['rate'] = np.where(records['rate'] < 4, 0, 1)

In [ ]:
records.head(15)

,uid,bid,rate
0,u1u,b2b,0
1,u1u,b29b,0
2,u1u,b32b,0
3,u1u,b47b,0
4,u1u,b50b,0
5,u1u,b112b,0
6,u1u,b151b,1
7,u1u,b223b,1
8,u1u,b253b,1
9,u1u,b260b,1


###saving
* binarized reviews csv

In [ ]:
records.to_csv(os.path.join(specific_version_data , "binarized_reviews.csv"), index=False)

#Split

divide the dataset between train, validation and test proportions

In [ ]:
records = pd.read_csv(os.path.join(specific_version_data , "binarized_reviews.csv"))
#records

explanation
* reserve 10000 users for test phase and 10000 for validation. the rest of users are only for training. 

* the VAE does a primary training run on this train and validation data

* with the test users, take 80% of records/reviews and form a secondary training set. the VAE does a secondary training run on the 80% test-training records. it must then be tested on ability to fill in test-testing records


In [ ]:
def split_users(unique_u_file, num_vad_u=10000, num_test_u=10000):
  with open(unique_u_file , 'r') as ufile:
    unique_u = ufile.readlines()
    unique_u = [u.strip() for u in unique_u]
  #NB it is already shuffled by the time it went INTO the file unique_u. no need to shuffle again
  test_u = unique_u[:num_test_u]
  vad_u = unique_u[num_test_u:num_test_u + num_vad_u]
  train_u = unique_u[num_test_u + num_vad_u:]

  return train_u, vad_u, test_u

In [ ]:
def split_df_on_users(original_DF, train_u, vad_u, test_u):
  train_rec = original_DF.where(original_DF['uid'].isin(train_u)).dropna()

  vad_rec =   original_DF.where(original_DF['uid'].isin(vad_u)  ).dropna()

  test_rec =  original_DF.where(original_DF['uid'].isin(test_u) ).dropna()
  
  return train_rec, vad_rec, test_rec

In [ ]:
#done this long way to allow progess to be measured in case of cutoff/crash
def split_test_DF(original_DF, wanted_test_proportion=0.2):

  #sort the original DF by the id user
  original_DF = original_DF.sort_values(['uid']).reset_index()

  tempList = []
  trainList = []
  testList = []

  prev_id_group = original_DF['uid'][0]
  for i in range(len(original_DF['uid'])):

    current_id_group = original_DF['uid'][i]
    current_record = [ original_DF['uid'][i], original_DF['bid'][i], original_DF['rate'][i] ]

    if current_id_group != prev_id_group: #have reached new group (user)
      #if elible for split over all groups 
      len_group = len(tempList)
      #if len_group >= min_user_interaction_threshold:
      #split the group over train test subsets
      random.shuffle(tempList)
      ts = int(len_group * wanted_test_proportion)
      testList.extend(tempList[:ts])
      trainList.extend(tempList[ts:])
      #else:
      #trainList.extend(tempList)   

      tempList = [] #clear templist to begin adding new group into it


    tempList.append(current_record) #add records to temp list. when new group(user) reached, split templist and clear it to start in new user
    prev_id_group = current_id_group

    #if not i%10000:
    #  print(i)

  tr_DF = pd.DataFrame(trainList, columns = ['uid','bid','rate'])
  ts_DF = pd.DataFrame(testList, columns = ['uid','bid','rate'])
  
  return tr_DF, ts_DF

In [ ]:
train_u, vad_u, test_u = split_users(os.path.join(specific_version_data,"unique_u.txt")) # lookups for user id for each Mutually exclusive set of users

In [ ]:
train_rec, vad_rec, test_rec = split_df_on_users(records, train_u, vad_u, test_u)
print(len(records['uid']))
print(len(train_rec['uid']))
print(len(vad_rec['uid']))
print(len(test_rec['uid']))


print(len(records['uid'].unique()))
print(len(train_rec['uid'].unique()))
print(len(vad_rec['uid'].unique()))
print(len(test_rec['uid'].unique()))


train_rec.head()

test_rec.shape

19984024
17098187
1454660
1431177
138493
118493
10000
10000


(1431177, 3)

In [ ]:
test_training_rec, test_testing_rec = split_test_DF(test_rec) #some test records are fed to algorithm to introduce it to the previously unseen users
# in bringing model up to speed on new test users, validate on same old validation data

###saving
* csv of train, test and validation records

In [ ]:
#keep user record lists as they serve as an int to str index of user ids
#test-train and test-test use the same index since the user indices are the same in both (u_test)
with open(os.path.join(split,"unique_u_train.txt") , 'w') as ufile:
  ufile.writelines('\n'.join(train_u))

with open(os.path.join(split,"unique_u_vad.txt") , 'w') as ufile:
  ufile.writelines('\n'.join(vad_u))

with open(os.path.join(split,"unique_u_test.txt") , 'w') as ufile: 
  ufile.writelines('\n'.join(test_u))


test_rec.to_csv(os.path.join(split,"test_rec.csv") , index=False)
vad_rec.to_csv(os.path.join(split,"vad_rec.csv") , index=False)
train_rec.to_csv(os.path.join(split,"train_rec.csv") , index=False)

test_training_rec.to_csv(os.path.join(split,"test_training_rec.csv") , index=False)
test_testing_rec.to_csv(os.path.join(split,"test_testing_rec.csv") , index=False)

#To Matrix
for the sake of the VAE, convert the records list format into a user-business matrix format.

In [ ]:
def records_to_matrix(recs_DF, u_mapping, b_mapping, implicit=True):
  #form numeric id row and col lists
  num_u = len(u_mapping.keys())
  num_b = len(b_mapping.keys())
  '''
  for i in recs_DF['uid']:
    print("df", type(i))
    break
  for i in u_mapping.keys():
    print("txt/map", type(i))
    break
  '''
  rows = [ u_mapping[uid] for uid in recs_DF['uid'] ] 
  cols = [ b_mapping[bid] for bid in recs_DF['bid'] ] 


  if implicit:
    values = np.ones_like(recs_DF['rate'])
  else:
    values = recs_DF['rate']
  sparse_mat = sparse.csr_matrix(  (values , (rows, cols)) , shape=( num_u,num_b) )

  return sparse_mat

In [ ]:
all_rec = pd.read_csv(os.path.join(specific_version_data,"binarized_reviews.csv"))
vad_rec = pd.read_csv(os.path.join(split,"vad_rec.csv"))
train_rec = pd.read_csv(os.path.join(split,"train_rec.csv"))
test_rec = pd.read_csv(os.path.join(split,"test_rec.csv"))
test_training_rec = pd.read_csv(os.path.join(split,"test_training_rec.csv"))
test_testing_rec = pd.read_csv(os.path.join(split,"test_testing_rec.csv"))


In [ ]:
all_rec.head()

,uid,bid,rate
0,u1u,b2b,0
1,u1u,b29b,0
2,u1u,b32b,0
3,u1u,b47b,0
4,u1u,b50b,0


In [ ]:
with open(os.path.join(specific_version_data,"unique_b.txt") , 'r', encoding='utf-8') as bfile:
  unique_b = bfile.readlines()
  unique_b = [b.strip() for b in unique_b]
  b_to_idx = {bid:i for (i, bid) in enumerate(unique_b)}

with open(os.path.join(specific_version_data,"unique_u.txt") , 'r') as ufile:
  u_to_idx = {uid:i for (i, uid) in enumerate([u.strip() for u in ufile.readlines()])}

with open(os.path.join(split,"unique_u_train.txt") , 'r') as ufile:
  u_train_to_idx = {uid:i for (i, uid) in enumerate([u.strip() for u in ufile.readlines()])}

with open(os.path.join(split,"unique_u_vad.txt") , 'r') as ufile:
  u_vad_to_idx = {uid:i for (i, uid) in enumerate([u.strip()for u in ufile.readlines()])}

with open(os.path.join(split,"unique_u_test.txt") , 'r') as ufile: 
  u_test_to_idx = {uid:i for (i, uid) in enumerate([u.strip() for u in ufile.readlines()])}


### Explicit Version:

In [ ]:
ex_all = records_to_matrix(all_rec,u_to_idx , b_to_idx ,implicit=False)
ex_train = records_to_matrix(train_rec, u_train_to_idx , b_to_idx ,implicit=False)
ex_vad = records_to_matrix(vad_rec, u_vad_to_idx , b_to_idx ,implicit=False)
ex_test = records_to_matrix(test_rec, u_test_to_idx , b_to_idx ,implicit=False)

ex_test_train = records_to_matrix(test_training_rec, u_test_to_idx , b_to_idx ,implicit=False)
ex_test_test = records_to_matrix(test_testing_rec, u_test_to_idx , b_to_idx ,implicit=False)

###Implicit Version

In [ ]:
im_all = records_to_matrix(all_rec,u_to_idx , b_to_idx ,implicit=True)
im_train = records_to_matrix(train_rec, u_train_to_idx , b_to_idx ,implicit=True)
im_vad = records_to_matrix(vad_rec, u_vad_to_idx , b_to_idx ,implicit=True)
im_test = records_to_matrix(test_rec, u_test_to_idx , b_to_idx ,implicit=True)

im_test_train = records_to_matrix(test_training_rec, u_test_to_idx , b_to_idx ,implicit=True)
im_test_test = records_to_matrix(test_testing_rec, u_test_to_idx , b_to_idx ,implicit=True)

###saving
* U-B sparse matrix for train, test and validation (implicit)
* U-B sparse matrix for train, test and validation (explicit)

In [ ]:
pickle.dump(im_all, open(os.path.join(implicit,"all_impl.pkl"), "wb"))
pickle.dump(im_train, open(os.path.join(implicit,"train_impl.pkl"), "wb"))
pickle.dump(im_vad, open(os.path.join(implicit,"vad_impl.pkl"), "wb"))

pickle.dump(im_test, open(os.path.join(implicit,"test_impl.pkl"), "wb"))
pickle.dump(im_test_train, open(os.path.join(implicit,"test_testing_impl.pkl"), "wb"))
pickle.dump(im_test_test, open(os.path.join(implicit,"test_training_impl.pkl"), "wb"))



pickle.dump(ex_all, open(os.path.join(explicit,"all_expl.pkl"), "wb"))
pickle.dump(ex_train, open(os.path.join(explicit,"train_expl.pkl"), "wb"))
pickle.dump(ex_vad, open(os.path.join(explicit,"vad_expl.pkl"), "wb"))

pickle.dump(ex_test, open(os.path.join(explicit,"test_expl.pkl"), "wb"))
pickle.dump(ex_test_train, open(os.path.join(implicit,"test_testing_impl.pkl"), "wb"))
pickle.dump(ex_test_test, open(os.path.join(implicit,"test_training_impl.pkl"), "wb"))

In [ ]:
im_vad.get_shape()

(10000, 127351)